# End-End RAG Backend Implementation

## Libraries

In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s et

In [2]:
!pip install pdfplumber transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.1 MB/s eta 0:00:00


In [7]:
import json
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer

In [26]:
import shutil
import pandas as pd

In [28]:
import os
from dotenv import load_dotenv

In [31]:
import google.generativeai as genai

## Document Preparation

In [8]:
json_path = "/content/uchicago_ads_scraped.json"

with open(json_path, "r", encoding="utf-8") as f:
    scraped_data = json.load(f)

print(f"Loaded {len(scraped_data)} pages.")

Loaded 66 pages.


In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
urls = list(scraped_data.keys())
texts = list(scraped_data.values())

In [12]:
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Generated embeddings shape: {embeddings.shape}")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Generated embeddings shape: (66, 384)


In [13]:
sample_idx = 0
print(f"URL: {urls[sample_idx]}\n")
print(f"Text sample: {texts[sample_idx][:500]}...\n")
print(f"Embedding vector (first 10 values):\n{embeddings[sample_idx][:10]}")

URL: https://datascience.uchicago.edu/news/can-a-doctors-notes-reveal-when-theyre-tired-new-research-illuminates-the-hidden-signals-of-physician-fatigue-and-raises-questions-about-ai-in-healthcare/

Text sample: Can a Doctor’s Notes Reveal When They’re Tired? New Research Illuminates the Hidden Signals of Physician Fatigue—And Raises Questions About AI in Healthcare – DSI Skip to main content About About the Data Science Institute The Data Science Institute (DSI) executes the University of Chicago’s bold, innovative vision of Data Science as a new discipline. Jobs & Opportunities Open faculty, postdoctoral, staff, and student roles with the UChicago Data Science Institute and our partners. Visiting DSI ...

Embedding vector (first 10 values):
[-0.02426697 -0.04705025  0.04663653  0.04374021  0.02563761 -0.07418401
 -0.09238268 -0.02648969 -0.0222806   0.05130934]


In [16]:
# combine one df with urls and texts
df = pd.DataFrame({
    'url': list(scraped_data.keys()),
    'text': list(scraped_data.values()),
    'embedding': list(embeddings)
})

print(df.head())

                                                 url  \
0  https://datascience.uchicago.edu/news/can-a-do...   
1         https://datascience.uchicago.edu/outreach/   
2  https://datascience.uchicago.edu/insights/nove...   
3  https://datascience.uchicago.edu/outreach/11th...   
4  https://datascience.uchicago.edu/events/ask-a-...   

                                                text  \
0  Can a Doctor’s Notes Reveal When They’re Tired...   
1  Outreach – DSI Skip to main content About Abou...   
2  Novel Intelligence Faculty Co-Director James E...   
3  11th Hour Project Software and Data Hub – DSI ...   
4  Ask A Student Information Session (Online Stud...   

                                           embedding  
0  [-0.024266968, -0.047050245, 0.046636533, 0.04...  
1  [-0.008831109, -0.066630505, 0.04877505, 0.059...  
2  [-0.035199653, -0.06787979, 0.0150821945, 0.04...  
3  [-0.029281419, -0.068970226, 0.03822872, 0.057...  
4  [-0.020103795, -0.06208864, 0.034671888, 0.053..

##  Store the Embeddings in a local Chroma vector database

In [18]:
# Create LangChain documents from your DataFrame
documents = [
    Document(
        page_content=row['text'],
        metadata={'source': row['url']}
    )
    for _, row in df.iterrows()
]

In [21]:
documents[0]

Document(metadata={'source': 'https://datascience.uchicago.edu/news/can-a-doctors-notes-reveal-when-theyre-tired-new-research-illuminates-the-hidden-signals-of-physician-fatigue-and-raises-questions-about-ai-in-healthcare/'}, page_content="Can a Doctor’s Notes Reveal When They’re Tired? New Research Illuminates the Hidden Signals of Physician Fatigue—And Raises Questions About AI in Healthcare – DSI Skip to main content About About the Data Science Institute The Data Science Institute (DSI) executes the University of Chicago’s bold, innovative vision of Data Science as a new discipline. Jobs & Opportunities Open faculty, postdoctoral, staff, and student roles with the UChicago Data Science Institute and our partners. Visiting DSI @ UChicago Contact Research Initiatives AI + Science A new paradigm of transformational AI-enabled scientific discovery across the physical and biological sciences. Data & Democracy Protecting democracy in the digital age through cross-disciplinary research an

In [22]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-412152783.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [25]:
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_uchicago_ads"
)


In [27]:
# make zip file to save for reuse later if needed
shutil.make_archive("/content/chroma_uchicago_ads", 'zip', "/content/chroma_uchicago_ads")


'/content/chroma_uchicago_ads.zip'

## Query Response Pipeline

### Load Gemini Model

In [30]:
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [32]:
genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel("gemini-pro")

### Create RAG

In [41]:
def RAG_pipeline_gemini(query, db, k=5):
    # Use the retriever to get top-k relevant chunks from Chroma
    # NOTE: The retriever automatically converts the query into an embedding using
    # the same encoder that was used for storing the document chunks.
    # No manual embedding needed.
    retriever = db.as_retriever(search_kwargs={"k": k})
    retrieved_docs = retriever.get_relevant_documents(query)

    # Combine the retrieved context into a single block of text
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

      # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

[CONTEXT]
{context}

[QUESTION]
{query}

Answer:"""

    model = genai.GenerativeModel("models/gemini-2.5-pro")

    response = model.generate_content(rag_prompt)

    # Print results + sources
    print("\n=== Question ===")
    print(query)
    print("\n=== Generated Answer ===")
    print(response.text)

    print("\n=== Sources ===")
    for doc in retrieved_docs:
        print(f"- {doc.metadata.get('source', 'No source available')}")

    return response.text


In [42]:
RAG_pipeline_gemini("What are the admission requirements for the UChicago program?", db)


=== Question ===
What are the admission requirements for the UChicago program?

=== Generated Answer ===
Based on the context provided, there is no information about the specific admission requirements for any of the University of Chicago's Data Science Institute (DSI) programs.

The documents describe the various academic programs offered, including:

*   **Undergraduate Data Science Major**
*   **Master's Programs** in computational analysis and public policy (MSCAPP) and applied data science (MS-ADS)
*   **PhD Program**

However, the texts do not detail the application process, prerequisites, GPA or testing requirements, or necessary application materials for these programs. The context does mention upcoming information sessions for the MS in Applied Data Science program, which would be a place to learn more about admissions.

=== Sources ===
- https://datascience.uchicago.edu/about/about-dsi/
- https://datascience.uchicago.edu/about/about-dsi/
- https://datascience.uchicago.edu/ne

"Based on the context provided, there is no information about the specific admission requirements for any of the University of Chicago's Data Science Institute (DSI) programs.\n\nThe documents describe the various academic programs offered, including:\n\n*   **Undergraduate Data Science Major**\n*   **Master's Programs** in computational analysis and public policy (MSCAPP) and applied data science (MS-ADS)\n*   **PhD Program**\n\nHowever, the texts do not detail the application process, prerequisites, GPA or testing requirements, or necessary application materials for these programs. The context does mention upcoming information sessions for the MS in Applied Data Science program, which would be a place to learn more about admissions."

In [44]:
RAG_pipeline_gemini("What are the core courses UChicago In-person ADS program?", db)


=== Question ===
What are the core courses UChicago In-person ADS program?

=== Generated Answer ===
Based on the context provided, the core courses for the University of Chicago's in-person Master of Science in Applied Data Science (MS-ADS) program are:

*   **Time Series Analysis and Forecasting**
*   **Statistical Models for Data Science**
*   **Machine Learning I**
*   **Machine Learning II**
*   **Data Engineering Platforms for Analytics** or **Big Data and Cloud Computing**
*   **Leadership and Consulting for Data Science**
*   **Data Science Capstone Project** (completed over two quarters)

=== Sources ===
- https://datascience.uchicago.edu/news/uchicago-ms-ads-students-present-capstone-project-at-midwest-ml-symposium/
- https://datascience.uchicago.edu/news/uchicago-ms-ads-students-present-capstone-project-at-midwest-ml-symposium/
- https://datascience.uchicago.edu/news/uchicago-alums-talk-careers-in-data-science-for-social-good-at-parliament-data/
- https://datascience.uchica

"Based on the context provided, the core courses for the University of Chicago's in-person Master of Science in Applied Data Science (MS-ADS) program are:\n\n*   **Time Series Analysis and Forecasting**\n*   **Statistical Models for Data Science**\n*   **Machine Learning I**\n*   **Machine Learning II**\n*   **Data Engineering Platforms for Analytics** or **Big Data and Cloud Computing**\n*   **Leadership and Consulting for Data Science**\n*   **Data Science Capstone Project** (completed over two quarters)"